# Семинар 5    
## Собираем поисковик 

![](https://bilimfili.com/wp-content/uploads/2017/06/bir-urune-emek-vermek-o-urune-olan-deger-algimizi-degistirir-mi-bilimfilicom.jpg) 


Мы уже все знаем, для того чтобы сделать поисковик. Осталось соединить все части вместе.    
Итак, для поисковика нам понадобятся:         
**1. База документов **
> в первом дз - корпус Друзей    
в сегодняшнем дз - корпус юридических вопросов-ответов    
в итоговом проекте - корпус Авито   

**2. Функция индексации**                 
Что делает: собирает информацию о корпусе, по которуму будет происходить поиск      
Своя для каждого поискового метода:       
> A. для обратного индекса она создает обратный индекс (чудо) и сохраняет статистики корпуса, необходимые для Okapi BM25 (средняя длина документа в коллекции, количество доков ... )             
> B. для поиска через word2vec эта функция создает вектор для каждого документа в коллекции путем, например, усреднения всех векторов коллекции       
> C. для поиска через doc2vec эта функция создает вектор для каждого документа               

   Не забывайте сохранить все, что насчитает эта функция. Если это будет происходить налету во время поиска, понятно, что он будет работать сто лет     
   
**3. Функция поиска**     
Можно разделить на две части:
1. функция вычисления близости между запросом и документом    
> 1. для индекса это Okapi BM25
> 2. для w2v и d2v это обычная косинусная близость между векторами          
2. ранжирование (или просто сортировка)


Время все это реализовать.

# Индексация
## Word2Vec
### Задание 1
Загрузите любую понравившуюся вам word2vec модель

In [1]:
from gensim.models import Word2Vec, KeyedVectors

In [153]:
# если модель без тэгов
w2v_model = Word2Vec.load('araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model')

In [2]:
# если модель с POS-тэггингом
model = KeyedVectors.load_word2vec_format('taiga_upos_skipgram_300_2_2018.vec', binary=False)

In [3]:
from nltk.corpus import stopwords

In [5]:
import nltk

In [10]:
from nltk.tokenize import word_tokenize

In [13]:
from string import punctuation

In [249]:
from pymystem3 import Mystem
mystem = Mystem()

In [20]:
punctuation+'»«–…'

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~»«–…'

### Задание 2 
Напишите функцию индексации для поиска через word2vec. Она должна для каждого документа из корпуса строить вектор.   
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только  вектор, но и опознователь текста, которому он принадлежит. 
Для поисковика это может быть url страницы, для поиска по текстовому корпусу сам текст.

> В качестве документа для word2vec берите **параграфы** исходного текста, а не весь текст целиком. Так вектора будут более осмысленными. В противном случае можно получить один очень общий вектор, релевантый совершенно разным запросам.

In [319]:
def preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
    1. lowercase, del punctuation, tokenize
    2. normal form
    3. del stopwords
    4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    words = [x.lower().strip(punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [mystem.lemmatize(x)[0] for x in words if x]

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

In [145]:
from judicial_splitter import split_paragraph, get_sentences

In [97]:
def get_paragraphs(file):
    with open(file, 'r', encoding='utf-8') as f:
        text = f.read() 
        paragraphs = split_paragraph(get_sentences(text), 4) 
        return paragraphs

In [316]:
def get_w2v_vectors(paragraph, model): 
    """
    Получает вектор документа
    (Так как в качестве документа нужно брать параграфы текста, данная функция получает вектор параграфа)
    """
    n = 0
    vector = [0] * 300
    lemmas = preprocessing(paragraph)
    for lemma in lemmas:
        try:
            vector += model.wv[lemma]
            n += 1
        except:
            None
    if n != 0:
        vector = vector / n
    return vector

def save_w2v_base(path, model):
    """Индексирует всю базу для поиска через word2vec"""
    w2v_base = []
    for file in os.listdir(path=path):
        for paragraph in get_paragraphs(path+'/'+file):
            vec = {}
            vec['id'] = file
            vec['text'] = paragraph
            vec['vector'] = get_w2v_vectors(paragraph, model)
            w2v_base.append(vec)
    return w2v_base


In [245]:
path = '/Users/kata/article_test'

In [142]:
import json

In [155]:
import tqdm
w2v_base = save_w2v_base(path, w2v_model)

In [156]:
w2v_base

[{'id': '1.txt',
  'text': '  Судья Высшего Арбитражного Суда Российской Федерации Ксенофонтова Н.А. рассмотрела заявление граждан Хорошавина Ю.Ф.,  Дробановой  С.Ю., Хорошавина С.Ю. от 14.12.2011 о пересмотре в порядке надзора решения Арбитражного суда Чувашской Республики от 25.08.2010 по делу N А79-5392/2010 и постановления Федерального арбитражного суда Волго-Вятского округа от 14.04.2011 по тому же делу по иску закрытого акционерного общества "Фирма "Чебоксарская керамика" (г.Чебоксары) к обществу с ограниченной ответственностью "Гарус" (г.Чебоксары) о взыскании 34 459 рублей 20 копеек задолженности по оплате коммунальных услуг.Суд установил: данное заявление не отвечает требованиям, предъявляемым к заявлению о пересмотре в порядке надзора судебных актов. В соответствии с частью 1 статьи 292 Арбитражного процессуального кодекса Российской  Федерации  вступившие в законную силу судебные акты арбитражных судов в Российской Федерации могут быть пересмотрены в порядке надзора Высшим А

## Doc2Vec
### Задание 3
Напишите функцию обучения doc2vec на юридических текстах, и получите свою кастомную d2v модель. 
> Совет: есть мнение, что для обучения doc2vec модели не нужно удалять стоп-слова из корпуса. Они являются важными семантическими элементами.      

Важно! В качестве документа для doc2vec берите **параграфы** исходного текста, а не весь текст целиком. И не забывайте про предобработку.

In [114]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from math import log

In [123]:
def get_tagged_data(path):
    tagged_data = []
    i = 0
    for file in os.listdir(path=path):
        for paragraph in get_paragraphs(path+'/'+file):
            data = preprocessing(paragraph, del_stopwords=False)
            tagged_data.append(TaggedDocument(words=data, tags=[i]))
            i+=1
    return tagged_data

In [125]:
def train_doc2vec(tagged_data):
    model = Doc2Vec(vector_size=100, min_count=5, alpha=0.025, min_alpha=0.025, epochs=100, workers=2, dm=1)
    model.build_vocab(tagged_data)
    model.random.seed(12345)
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    
    return model

In [127]:
d2w_model = train_doc2vec(get_tagged_data(path))

### Задание 4
Напишите функцию индексации для поиска через doc2vec. Она должна для каждого документа из корпуса получать вектор.    
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только вектор, но и опознователь текста, которому он принадлежит. 

In [315]:
def get_d2v_vectors(text, model):
    """Получает вектор документа"""
    return model.infer_vector(text)

def save_d2v_base(path, model):
    """Индексирует всю базу для поиска через doc2vec"""
    d2v_base = []
    for file in os.listdir(path=path):
        for paragraph in get_paragraphs(path+'/'+file):
            vec = {}
            vec['id'] = file
            vec['text'] = paragraph
            vec['vector'] = get_d2v_vectors(paragraph, model)
            d2v_base.append(vec)
    return d2v_base

In [134]:
d2v_base = save_d2v_base(path, d2w_model)

# Функция поиска

Для обратного индекса функцией поиска является Okapi BM25. Она у вас уже должна быть реализована.

Функция измерения близости между векторами нам пригодится:

In [144]:
from gensim import matutils
import numpy as np 

def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)

### Okapi

In [178]:
import math
from math import log

In [320]:
from collections import Counter
import numpy as np

In [271]:
def get_information(path):
    """
    Create inverted index by input doc collection
    Get essential information from input doc collection
    :return: inverted index, 
    """
    dictionary = {}
    files_len = {}
    for file in os.listdir(path):
        with open(path +'/'+ file, 'r', encoding='utf-8') as f:
            text = f.read()
            words = preprocessing(text)
            files_len[file] = len(words)
            counter = Counter(words)
            for word in counter:
                if word in dictionary:
                    dictionary[word][file] = counter[word]
                else:
                    dictionary[word] = {}
                    dictionary[word][file] = counter[word]
    return files_len, dictionary

In [270]:
files_len, index = get_information(path)

In [268]:
index

{'судья': {'1.txt': 2,
  '10.txt': 3,
  '100.txt': 5,
  '11.txt': 3,
  '12.txt': 3,
  '13.txt': 3,
  '14.txt': 3,
  '15.txt': 3,
  '16.txt': 4,
  '17.txt': 3,
  '18.txt': 2,
  '19.txt': 3,
  '2.txt': 2,
  '20.txt': 3,
  '21.txt': 3,
  '22.txt': 3,
  '23.txt': 5,
  '24.txt': 4,
  '25.txt': 3,
  '26.txt': 3,
  '27.txt': 7,
  '28.txt': 7,
  '29.txt': 3,
  '3.txt': 7,
  '30.txt': 3,
  '31.txt': 3,
  '32.txt': 3,
  '33.txt': 2,
  '34.txt': 5,
  '35.txt': 3,
  '36.txt': 7,
  '37.txt': 3,
  '38.txt': 3,
  '39.txt': 5,
  '4.txt': 3,
  '40.txt': 3,
  '41.txt': 6,
  '42.txt': 3,
  '43.txt': 3,
  '44.txt': 5,
  '45.txt': 6,
  '46.txt': 6,
  '47.txt': 6,
  '48.txt': 6,
  '49.txt': 3,
  '5.txt': 4,
  '50.txt': 5,
  '51.txt': 3,
  '52.txt': 6,
  '53.txt': 3,
  '54.txt': 3,
  '55.txt': 3,
  '56.txt': 2,
  '57.txt': 6,
  '58.txt': 3,
  '59.txt': 3,
  '6.txt': 7,
  '60.txt': 6,
  '61.txt': 3,
  '62.txt': 3,
  '63.txt': 6,
  '64.txt': 3,
  '65.txt': 3,
  '66.txt': 3,
  '67.txt': 3,
  '68.txt': 6,
  '69.

In [272]:
files_len

{'1.txt': 228,
 '10.txt': 107,
 '100.txt': 130,
 '11.txt': 114,
 '12.txt': 128,
 '13.txt': 113,
 '14.txt': 159,
 '15.txt': 88,
 '16.txt': 229,
 '17.txt': 221,
 '18.txt': 103,
 '19.txt': 155,
 '2.txt': 260,
 '20.txt': 158,
 '21.txt': 121,
 '22.txt': 99,
 '23.txt': 87,
 '24.txt': 182,
 '25.txt': 150,
 '26.txt': 100,
 '27.txt': 144,
 '28.txt': 144,
 '29.txt': 120,
 '3.txt': 125,
 '30.txt': 121,
 '31.txt': 121,
 '32.txt': 115,
 '33.txt': 301,
 '34.txt': 113,
 '35.txt': 100,
 '36.txt': 99,
 '37.txt': 99,
 '38.txt': 108,
 '39.txt': 104,
 '4.txt': 144,
 '40.txt': 108,
 '41.txt': 208,
 '42.txt': 106,
 '43.txt': 109,
 '44.txt': 100,
 '45.txt': 250,
 '46.txt': 250,
 '47.txt': 246,
 '48.txt': 246,
 '49.txt': 109,
 '5.txt': 503,
 '50.txt': 294,
 '51.txt': 154,
 '52.txt': 200,
 '53.txt': 274,
 '54.txt': 267,
 '55.txt': 175,
 '56.txt': 272,
 '57.txt': 156,
 '58.txt': 108,
 '59.txt': 91,
 '6.txt': 100,
 '60.txt': 260,
 '61.txt': 217,
 '62.txt': 116,
 '63.txt': 111,
 '64.txt': 92,
 '65.txt': 112,
 '66

In [310]:
k1 = 2.0
b = 0.75

def score_BM25(qf, dl, avgdl, k1, b, N, n) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    score = math.log((N-n+0.5)/(n+0.5)) * (k1+1)*qf/(qf+k1*(1-b+b*(dl/avgdl)))
    return score

In [312]:
def compute_sim(word, index, files_len):
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    N = len(files_len)
    avgdl = sum(files_len.values())/N
    if word in index:
        n = len(index[word])
        result = {}
        for file in index[word]:
            qf = index[word][file]
            score = score_BM25(qf, files_len[file], avgdl, k1, b, N, n)
            result[file] = score
        return result
    else:
        return {}


def get_search_result(inquiry):
    """
    Compute sim score between search query and all documents in collection
    :return: list of files
    """
    global index, files_len
    score = defaultdict(int)
    words = preprocessing(inquiry)
    for word in words:
        result = compute_sim(word, index, files_len)
        for file in result:
            score[file] += result[file]  
        
    return sorted(score, key=score.get)[:10]

In [308]:
from collections import defaultdict
get_search_result('суд')

['83.txt',
 '55.txt',
 '94.txt',
 '51.txt',
 '95.txt',
 '3.txt',
 '53.txt',
 '60.txt',
 '45.txt',
 '46.txt']

### Задание 5
Напишите функцию для поиска через word2vec и для поиска через doc2vec, которая по входящему запросу выдает отсортированную выдачу документов.

In [313]:
import math
def search_w2v(inquiry, base, model):
    v1 = get_w2v_vectors(inquiry, model)
    score = defaultdict(int)
    for vec in base:
        v2 = vec['vector']
        sim = similarity(v1, v2)
        score[vec['id']] = sim
    return sorted(score, key=score.get)[:10]
        

def search_d2v(inquiry, base, model):
    v1 = get_d2v_vectors(inquiry, model)
    score = defaultdict(int)
    for vec in base:
        v2 = vec['vector']
        sim = similarity(v1, v2)
        score[vec['id']] = sim
    return sorted(score, key=score.get)[:10]

In [317]:
search_w2v('о признании недействительным предписания по устранению выявленных нарушений', w2v_base, w2v_model)

/Users/kata/anaconda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


['74.txt',
 '16.txt',
 '2.txt',
 '24.txt',
 '4.txt',
 '53.txt',
 '73.txt',
 '77.txt',
 '99.txt',
 '100.txt']

In [318]:
search_d2v('о признании недействительным предписания по устранению выявленных нарушений', d2v_base, d2w_model)

/Users/kata/anaconda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


['2.txt',
 '73.txt',
 '74.txt',
 '24.txt',
 '4.txt',
 '53.txt',
 '99.txt',
 '16.txt',
 '77.txt',
 '100.txt']

После выполнения всех этих заданий ваш поисковик готов, поздравляю!                  
Осталось завернуть все написанное в питон скрипт, и сделать общую функцию поиска гибким, чтобы мы могли искать как по обратному индексу, так и по word2vec, так и по doc2vec.          
Сделать это можно очень просто через старый добрый ``` if ```, который будет дергать ту или иную функцию поиска:

In [ ]:
def search(..., search_method):
    if search_method == 'inverted_index':
        ...
        search_result = search_inv_index()
    elif search_method == 'word2vec':
        ...
        search_result = search_w2v()
    elif search_method == 'doc2vec':
        ...
        search_result = search_d2v()
    else:
        raise TypeError('unsupported search method')